# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

# 1. QuickView of VisionKG

## 1.1. One-Click to meet VisionKG

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import output
# install our vision api
!python -m pip install git+https://github.com/cqels/vision.git --force
output.clear()

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
# import SemkgAPI
from vision_utils import semkg_api, data
from IPython.display import Image, display

query_string='''#Give me the images containing person and cat
prefix cv:<http://vision.semkg.org/onto/v0.1/>
SELECT DISTINCT ?image
WHERE {
    ?ann1 a cv:Annotation.
    ?ann1 cv:isAnnotationOfImage ?image.
    ?ann1 cv:hasAnnotatedObject ?obj1.
    ?obj1 cv:hasLabel "person".
    ?ann2 a cv:Annotation.
    ?ann2 cv:isAnnotationOfImage ?image.
    ?ann2 cv:hasAnnotatedObject ?obj2.
    ?obj2 cv:hasLabel "cat".
    ?image cv:hasLocalPath ?localPath.
}
LIMIT 100'''

result=semkg_api.query(query_string)

print("Sample images from the query result")
for i in range(3):
    image = result['images'][i]
    display(Image(semkg_api.SEMKG_IMAGES_HOST + image['image_path'] , width=400))

# 2. Object Detection in Practice starting from VisionKG

##2.1. VisionKG meet mmdetection

In [ ]:
# clone the toolbox for object detection
!git clone https://github.com/jichengyuan/mmdetectionCust.git

# install dependencies
%cd mmdetectionCust/
!python -m pip install cython funcy && python -m pip install -r requirements.txt
!python -m pip install mmcv-full==1.3.14 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
!python setup.py develop

output.clear()

## 2.2 Load your customized Dataset from local Disk

In [ ]:
import json
from os.path import join as opj
from utils import check_download_images
# save query result
path_to_anno_mixedDatasets = opj('data/mixedDatasets/',
                                 'test_query_api_image.json')
with open(path_to_anno_mixedDatasets, "w") as f:
    json.dump(result,f)
# check images queried from VisionKG
check_download_images(result["images"])

## 2.3 Perform Training & Evaluation on your chosen Object Detection tool

In [ ]:
from utils import dataset_split, checkpoint_verify

# Split the quried MixedDatasets and check the #instances per category
ratio = 0.8
train_val_test = ['train.json','val.json','test.json']
dataset_split(path_to_anno_mixedDatasets, train_val_test, ratio)
nms_categories = [category['name'] for category in result['categories']]
num_categories = len(nms_categories)

# Set paths for config and work-dir
path_to_config = 'configs/fcos/fcos_visionKG.py'
path_to_work_dir = 'mixedDatasets/logs_visionKG/'

# Training based on the queried MixedDataset
# For more params-setting, please check: 
# https://mmdetection.readthedocs.io/en/latest/
%run tools/train.py {path_to_config} \
--work-dir {path_to_work_dir} \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.train.classes="$nms_categories" data.val.classes="$nms_categories"

# Verify the checkpoint file.
checkpoint_file = checkpoint_verify(path_to_work_dir)

# Evaluate the trained model on the MixedDataset
%run tools/test.py {path_to_config} {checkpoint_file} \
--work-dir {path_to_work_dir} \
--cfg-options model.bbox_head.num_classes={num_categories} \
data.test.classes="$nms_categories" --eval bbox --show